<a href="https://colab.research.google.com/github/fasthill/ML-DL-study-alone/blob/main/5-1%20%EA%B2%B0%EC%A0%95%20%ED%8A%B8%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## calculate prediction accuracy

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

In [1]:
import numpy as np
import pandas as pd
import datetime
import joblib
import os, re

In [4]:
import pickle

# write list, dictionary to pickle
def save_to_pickle(path, filename):
    open_file = open(path, "wb")
    pickle.dump(filename, open_file)
    open_file.close()

# read list, dictionary from pickle
def load_from_pickle(path):
    open_file = open(path, "rb")
    loaded_file = pickle.load(open_file)
    open_file.close()
    return loaded_file

In [5]:
# import csv

In [8]:
def is_new_format(f_name): #5월 16일 이전 생성 데이터 확인
    mon = int(f_name[11:13])
    day = int(f_name[13:15])
    if (mon < 5):
        return False
    elif (mon == 5) & (day < 16) :
        return False
    else:
        return True

In [9]:
def find_filename(dir): # find a filename in a directory
    p = re.compile('lgbm_bs_df_(.{9}).*csv')
    dir_list = os.listdir(dir)
    for fname in dir_list:
        aa = p.search(fname)
        if aa is None:
            continue
        else:
            return aa.group()

In [13]:
def find_next_date(current_date):
    current_index = list(OPENING_DAYS_KOR).index(current_date)
    next_date =  OPENING_DAYS_KOR.iloc[current_index+1]
    return next_date

In [28]:
def calc_result_df(start_id, end_id):
    predict_value = {}
    p0 = []
    p1 = []
    p2 = []
    p3 = []
    p4 = []
    p5 = []
    p6 = []
    p7 = []
    p8 = []
    p9 = []
    p_date = []

    n_count = 0 # 총 갯수
    np_count = 0
    # prediction_date = next_date
    for prediction_date in OPENING_DAYS_KOR[st_id:end_id]:
        prediction_row = current_data[current_data.index == prediction_date]
        if(len(prediction_row) == 0):
#             print("**** No data on", prediction_date, "*****")
            continue
    #     print(prediction_row.iloc[:, 0:5])
        n_count = n_count + 1
        com_row = com_data[com_data['date']  == prediction_date]

        # predict on,off low price, high price
        real_data_df = prediction_row[real_columns] # select necessary columns
        real_scaled = scaler.transform(real_data_df)

        y_predict = model.predict(real_scaled)
        weight = model.predict_proba(real_scaled)

        if y_predict[0] == 1:
            np_count = np_count + 1
    #         print('prediction data: ', prediction_date, 'prediction result :', y_predict[0])
            ratio = com_row['close_cr'].iloc[0]
            if ratio >= 0: # 전일 대비 + 비교
                t_f = True
            else:
                t_f = False

            p0.append(com_name)
            p1.append(y_predict[0])
            p2.append(t_f)
            p3.append(ratio)
            p_value = com_row['close'].iloc[0] / (1 + ratio/100.)
            p4.append(f'{p_value:.1f}')
            p_c_value = p_value * (1 - 0.005) #  구매 기준: 0.005% 이하로 떨어지면 구매 가능
            p5.append(f'{p_c_value:.1f}') # 0.5% 적용
            p6.append(com_row['low'].iloc[0])
            p7.append(f"{(p_c_value - com_row['low'].iloc[0]):.1f}")
            if p_c_value >= com_row['low'].iloc[0] :
                p8_yesno = 'yes'
            else:
                p8_yesno = 'no'
            p8.append(p8_yesno)
            if (p8_yesno == 'yes') & t_f :
                final = 'yes'
            else:
                final = 'no'
            p9.append(final)
            p_date.append(prediction_date)

    predict_value['date'] = p_date
    predict_value['company'] = p0
    predict_value['prediction'] = p1
    predict_value['result'] = p2
    predict_value['cr_ratio'] = p3
    predict_value['p_close'] = p4
    predict_value['p_c_05%'] = p5
    predict_value['low'] = p6
    predict_value['diff'] = p7
    predict_value['chance_to_buy'] = p8
    predict_value['final'] = p9  # 최종 취득 가능 판단

#     pre_ratio = np_count / n_count
#     print(f"Count total: {n_count}, count_yes: {np_count}, ratio: {pre_ratio:.2f}")
    result_df = pd.DataFrame(predict_value)
    
    return result_df

In [10]:
import sys, os

module_path = os.path.abspath(os.path.join('.')) # 현재 폴더로 이동
if module_path+"\\data\\base_data\\common_data" not in sys.path:
    sys.path.append(module_path+"\\data\\base_data\\common_data") #  공통으로 사용하는 각종 리스트, 코드 등 
    
import common_data as cd

In [11]:
# get stock market opening days
base_data_directory = './data/base_data/stock_market_holydays/'
OPENING_DAYS_KOR = pd.read_pickle(base_data_directory+'opening_days_kor.pkl') # 한국 개장일 데이터 
OPENING_DAYS_USA = pd.read_pickle(base_data_directory+'opening_days_usa.pkl') # 미국 개장일 데이터 

In [12]:
TRAIN_START_DATE = datetime.date(2022, 3, 2)
TRAIN_END_DATE = datetime.date(2023, 3, 31)

In [15]:
directory_for_predict = './data/data_for_ml/predict/'
directory_for_data = './data/company_pkl/'
directory_for_common = './data/common_pkl/'
directory_for_model = './data/data_for_ml/model/model/'

In [21]:
code_good = {'005930': ['삼성전자', 'sec'], '035420': ['NAVER', 'naver'],
             '005380': ['현대차', 'hyunmotor'], '035720': ['카카오', 'kakao'],
             '000660': ['SK하이닉스', 'skhynix'], '006400': ['삼성SDI', 'sdi'],
             '005935': ['삼성전자우', 'secpre'], '105560': ['KB금융', 'kbbank'],
             '012330': ['현대모비스', 'mobis'],  '055550': ['신한지주', 'shgroup'],
             '003670': ['포스코퓨처엠', 'poscochemical'], '034730': ['SK', 'sk'], 
             '032830': ['삼성생명', 'sslife'], '086790': ['하나금융지주', 'hana'],
             '009150': ['삼성전기', 'sselec'], '017670': ['SK텔레콤', 'sktelecom'],
             '011200': ['HMM', 'hmm'], '000810': ['삼성화재', 'ssfire'], 
             '010950': ['S-Oil', 'soil'], '018260': ['삼성에스디에스', 'sds'],
             '316140': ['우리금융지주', 'woorifg'], '024110': ['기업은행', 'ibk'], 
             '377300': ['카카오페이', 'kakaopay'], '028050': ['삼성엔지니어링', 'ssengineering'],
            }

In [17]:
# com_name = 'poscochemical'
TRAIN_START_DATE = datetime.date(2022, 3, 2)
TRAIN_END_DATE = datetime.date(2023, 3, 31)

In [38]:
df_base = pd.DataFrame()

for key, val in code_good.items():
 
    com_name = val[1]
    
    # read selected columns, scaler and model to be appllied

    # locate the model data directory
    directory_model_data = f'{directory_for_model}{com_name}/'

    # get the model data filepath
    columns_pkl = directory_model_data + 'best_columns.pkl' # save with pickle.dump 
    scaler_pkl = directory_model_data + 'best_scaler.pkl'  # one saved with joblib.dump
    scaler_p_pkl = directory_model_data + 'best_scaler_p.pkl' # one saved with pickle.dump
    model_pkl = directory_model_data + 'best_model.pkl'  # one saved with joblib.dump
    model_p_pkl = directory_model_data + 'best_model_p.pkl' # one saved with pickle.dump
    result_pkl = directory_model_data + 'best_result.pkl'

    # load iterating training result data
    result = load_from_pickle(result_pkl) # with pickle.load

    yes_no = is_new_format(find_filename(directory_model_data))
    if yes_no:
        precision = result.loc['test_precision'].iloc[-1]
    else:
        precision = result.loc['precision'].iloc[-1]
        
    # load columns data
    real_columns = load_from_pickle(columns_pkl)[:-5] # column 읽기. target columns 5개는 제외
    # scale the data
    scaler = joblib.load(scaler_pkl) # scaler 읽기
    # apply the scaled real_data to the model
    model = joblib.load(model_pkl) # model 읽기
    
    fname = f'df_{com_name}_combine.pkl'
    f_name = directory_for_predict + fname
    df_o = pd.read_pickle(f_name) 
    com_fname = f'{com_name}_historical.pkl'  # 실제와 예측을 비교하기 위하여 실제데이터을 불러 옴
    f_com_name = directory_for_data + com_fname
    com_data = pd.read_pickle(f_com_name)
    com_data['date'] = com_data['date'].dt.date
    
    current_data = df_o.loc[:, 'retail_1':'weekday'] # select columns except targets columns
    
    next_date = find_next_date(TRAIN_END_DATE) # 최초 시작날짜 설정 (훈련데이터 이후 첫째 날짜)
    
    st_id = list(OPENING_DAYS_KOR).index(next_date) # 훈련데이터 이후 첫째 날짜 index == text data 첫째 날짜
    end_id = list(OPENING_DAYS_KOR).index(df_o.index[-1]) # test data 마지막 날짜
    
    df_com = calc_result_df(st_id, end_id)
    
    df_base = pd.concat([df_base, df_com],axis=0)

In [39]:
df_base

,date,company,prediction,result,cr_ratio,p_close,p_c_05%,low,diff,chance_to_buy,final
0,2023-04-03,sec,1,False,-1.41,64002.4,63682.4,63000.0,682.4,yes,no
1,2023-04-18,sec,1,True,0.46,65299.6,64973.1,64800.0,173.1,yes,yes
2,2023-04-28,sec,1,True,1.39,64602.0,64279.0,65000.0,-721.0,no,no
3,2023-05-08,sec,1,True,1.23,65099.3,64773.8,65400.0,-626.2,no,no
4,2023-05-16,sec,1,True,1.40,64497.0,64174.6,65300.0,-1125.4,no,no
...,...,...,...,...,...,...,...,...,...,...,...
2,2023-04-14,ssengineering,1,True,1.34,29948.7,29798.9,30250.0,-451.1,no,no
3,2023-05-08,ssengineering,1,True,2.63,28500.4,28357.9,28550.0,-192.1,no,no
4,2023-06-07,ssengineering,1,False,-1.36,29349.1,29202.4,28850.0,352.4,yes,no
5,2023-06-08,ssengineering,1,False,-1.21,28950.3,28805.5,28500.0,305.5,yes,no


In [43]:
df_base['result'].value_counts()

result
True    32
Name: count, dtype: int64

In [42]:
df_base = df_base[df_base['final'] == 'yes']

In [44]:
df_base.sort_values(by=['date']).reset_index(drop=True)

,date,company,prediction,result,cr_ratio,p_close,p_c_05%,low,diff,chance_to_buy,final
0,2023-04-03,kakaopay,1,True,0.18,55599.9,55321.9,55000.0,321.9,yes,yes
1,2023-04-03,sdi,1,True,1.36,735003.9,731328.9,730000.0,1328.9,yes,yes
2,2023-04-04,ssengineering,1,True,1.13,31098.6,30943.1,30750.0,193.1,yes,yes
3,2023-04-04,sktelecom,1,True,0.32,47547.8,47310.1,47000.0,310.1,yes,yes
4,2023-04-07,poscochemical,1,True,1.20,292490.1,291027.7,287500.0,3527.7,yes,yes
5,2023-04-12,sds,1,True,0.43,116996.9,116411.9,116200.0,211.9,yes,yes
6,2023-04-18,ibk,1,True,0.00,10120.0,10069.4,10010.0,59.4,yes,yes
7,2023-04-18,woorifg,1,True,0.78,11569.8,11511.9,11470.0,41.9,yes,yes
8,2023-04-18,sec,1,True,0.46,65299.6,64973.1,64800.0,173.1,yes,yes
9,2023-04-19,hyunmotor,1,True,1.15,191398.9,190441.9,190300.0,141.9,yes,yes


In [ ]:
code = cd.code_all # 전체 회사 코드

code_mid = {'373220': ['LG에너지솔루션', 'lgenergy'], '207940': ['삼성바이오로직스', 'ssbio'],
            '000270': ['기아', 'kia'], '028260': ['삼성물산', 'sscnt'],
            '015760': ['한국전력', 'koreaelec'], '034020': ['두산에너빌리티', 'doosanener'],
            '051900': ['LG생활건강', 'lglife'], '259960': ['크래프톤', 'crafton'],
            '361610': ['SK아이이테크놀로지', 'skietech'], '086280': ['현대글로비스', 'glovis'],
            '302440': ['SK바이오사이언스', 'skbio'],
            }

code_bad = {'051910': ['LG화학', 'lgchemical'], '033780': ['KT&G', 'ktng'],
            '005490': ['POSCO홀딩스', 'poscoholding'], '068270': ['셀트리온', 'celltrion'],
            '066570': ['LG전자', 'lgelec'],  '096770': ['SK이노베이션', 'skinnovation'],
            '030200': ['KT', 'kt'], '003550': ['LG', 'lg'],
            '329180': ['현대중공업', 'hhi'], '003490': ['대한항공', 'koreanair'],
            '036570': ['엔씨소프트', 'ncsoft'], '009830': ['한화솔루션', 'hanhwasol'],
            '090430': ['아모레퍼시픽', 'amore'], '011170': ['롯데케미칼', 'lottechem'],
            '138040': ['메리츠금융지주', 'meritz'], '011070': ['LG이노텍', 'lginnotek'],
           }

code_good = {'005930': ['삼성전자', 'sec'], '035420': ['NAVER', 'naver'],
             '005380': ['현대차', 'hyunmotor'], '035720': ['카카오', 'kakao'],
             '000660': ['SK하이닉스', 'skhynix'], '006400': ['삼성SDI', 'sdi'],
             '005935': ['삼성전자우', 'secpre'], '105560': ['KB금융', 'kbbank'],
             '012330': ['현대모비스', 'mobis'],  '055550': ['신한지주', 'shgroup'],
             '003670': ['포스코퓨처엠', 'poscochemical'], '034730': ['SK', 'sk'], 
             '032830': ['삼성생명', 'sslife'], '086790': ['하나금융지주', 'hana'],
             '009150': ['삼성전기', 'sselec'], '017670': ['SK텔레콤', 'sktelecom'],
             '011200': ['HMM', 'hmm'], '000810': ['삼성화재', 'ssfire'], 
             '010950': ['S-Oil', 'soil'], '018260': ['삼성에스디에스', 'sds'],
             '316140': ['우리금융지주', 'woorifg'], '024110': ['기업은행', 'ibk'], 
             '377300': ['카카오페이', 'kakaopay'], '028050': ['삼성엔지니어링', 'ssengineering'],
            }